## Perform power analysis given observed linear model effect sizes

In [1]:
library(pwr)
suppressPackageStartupMessages(library(dplyr))

In [2]:
plate <- "localhost220513100001_KK22-05-198_FactinAdjusted"

lm_results_file <- file.path("results", paste0(plate, "_linear_model_cp_features.tsv"))
output_file <- file.path("results", paste0(plate, "_power_analysis_cp_features_lm.tsv"))

In [3]:
# Load linear model results
lm_results_df <- readr::read_tsv(
    lm_results_file,
    col_types = readr::cols(.default="d", feature="c")
)

print(dim(lm_results_df))
head(lm_results_df)

[1] 585   4


feature,r2_score,cell_count_coef,treatment_dose_coef
<chr>,<dbl>,<dbl>,<dbl>
Cytoplasm_AreaShape_Compactness,0.04421516,-2.511474e-05,-0.07800511
Cytoplasm_AreaShape_Extent,0.06927086,-7.737930e-05,0.09629138
Cytoplasm_AreaShape_FormFactor,0.09534301,-1.595287e-04,0.11192168
Cytoplasm_AreaShape_MajorAxisLength,0.08323591,-1.751328e-03,0.01435686
Cytoplasm_AreaShape_Perimeter,0.04565131,-1.368908e-03,-0.01115763
Cytoplasm_AreaShape_Solidity,0.10653768,-1.835768e-04,0.11806437


In [4]:
# Load feature data (for calculating n)
file_suffix = "_sc_norm_fs_cellprofiler.csv.gz"
data_dir = file.path("..", "..", "..", "3.process-cfret-features", "data")
cp_file <- file.path(data_dir, paste0(plate, file_suffix))

cp_df <- readr::read_csv(
    cp_file,
    col_types = readr::cols(
        .default="d",
        Metadata_WellRow="c",
        Metadata_WellCol="c",
        Metadata_heart_number="c",
        Metadata_treatment="c",
        Metadata_dose="c",
        Metadata_Plate="c",
        Metadata_Well="c"
    )
)

print(dim(cp_df))
head(cp_df, 3)

[1] 17352   628


Metadata_WellRow,Metadata_WellCol,Metadata_heart_number,Metadata_treatment,Metadata_dose,Metadata_ImageNumber,Metadata_Plate,Metadata_Well,Metadata_Cytoplasm_Parent_Cells,Metadata_Cytoplasm_Parent_Nuclei,⋯,Nuclei_Texture_InverseDifferenceMoment_Mitochondria_3_01_256,Nuclei_Texture_InverseDifferenceMoment_Mitochondria_3_03_256,Nuclei_Texture_SumEntropy_ER_3_03_256,Nuclei_Texture_SumEntropy_Golgi_3_01_256,Nuclei_Texture_SumEntropy_Mitochondria_3_03_256,Nuclei_Texture_SumVariance_Actin_3_01_256,Nuclei_Texture_SumVariance_ER_3_03_256,Nuclei_Texture_SumVariance_Golgi_3_01_256,Nuclei_Texture_SumVariance_Hoechst_3_01_256,Nuclei_Texture_SumVariance_Mitochondria_3_03_256
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
A,9,9,drug_x,5uM,1,localhost220513100001,A09,1,3,⋯,-1.341714,-0.7714563,0.3830138,1.5279864,1.6033489,-0.4200762,-0.2362416,0.20380578,-0.3547300,0.16163561
A,9,9,drug_x,5uM,1,localhost220513100001,A09,2,4,⋯,1.211139,0.9654590,-0.9223457,-0.4915354,-1.0004204,-0.4608149,-0.4684389,-0.20632511,-0.3310333,-0.18902928
A,9,9,drug_x,5uM,1,localhost220513100001,A09,3,7,⋯,-1.117919,-1.4119451,0.1231898,0.9340863,0.9589492,-0.4689575,-0.2375866,-0.03696811,-0.1078189,-0.06320025


## Perform power analysis

In [5]:
# Define constants for power analysis
n_conditions <- length(table(cp_df$Metadata_dose))  # The number of doses
n_samples <- dim(cp_df)[1]
alpha_standard <- 0.05

u <- n_conditions - 1
v <- n_samples - u - 1
sig_level <- alpha_standard / dim(lm_results_df)[1]  # Multiple test adjusted
power <- 0.8

print(c(u, v))
print(sig_level)

[1]     9 17342
[1] 8.547009e-05


In [6]:
# Given all R2 values perform power analysis
all_power_results <- list()
for (cp_feature in lm_results_df$feature) {
    # Subset to the given feature lm results
    lm_result_subset_df <- lm_results_df %>%
        dplyr::filter(feature == !!cp_feature)
    
    # Pull out the estimated R2 value
    r2_val <- lm_result_subset_df %>% dplyr::pull(r2_score)
    
    # The power estimate is undefined for r2_val = 1, skip if so
    if (r2_val == 1) {
        all_power_results[[cp_feature]] <- c(cp_feature, u, v, sig_level, NULL, NULL)
        next
    }
    
    # Transform R2 score to F2 effect size
    f2_val <- r2_val / (1 - r2_val)
    
    # Calculate power, note that v contains an estimate of sample size
    power_result <- pwr.f2.test(u = u, v = NULL, f2 = f2_val, sig.level = sig_level, power = power)
    
    # Calculate required sample size from the v formula
    estimated_sample_size <- power_result$v + u + 1
    
    # Save results for future visualization
    all_power_results[[cp_feature]] <- c(cp_feature, u, v, sig_level, power, estimated_sample_size)
    
}

In [7]:
# Compile and output results
power_results_df <- do.call(rbind, all_power_results) %>% dplyr::as_tibble()

colnames(power_results_df) <- c("feature", "u", "v", "sig_level", "power", "estimated_sample_size")

# Output to file
power_results_df %>%
    readr::write_tsv(output_file)

print(dim(power_results_df))
head(power_results_df)

Warning message:
“The `x` argument of `as_tibble.matrix()` must have unique column names if `.name_repair` is omitted as of tibble 2.0.0.
ℹ Using compatibility `.name_repair`.”


[1] 585   6


feature,u,v,sig_level,power,estimated_sample_size
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Cytoplasm_AreaShape_Compactness,9,17342,8.54700854700855e-05,0.8,796.167165604314
Cytoplasm_AreaShape_Extent,9,17342,8.54700854700855e-05,0.8,501.335706879138
Cytoplasm_AreaShape_FormFactor,9,17342,8.54700854700855e-05,0.8,359.066202160395
Cytoplasm_AreaShape_MajorAxisLength,9,17342,8.54700854700855e-05,0.8,414.046782803633
Cytoplasm_AreaShape_Perimeter,9,17342,8.54700854700855e-05,0.8,770.524134001481
Cytoplasm_AreaShape_Solidity,9,17342,8.54700854700855e-05,0.8,319.348975542797
